# Install Java

In [ ]:
!sudo apt-get update
!sudo apt-get install openjdk-11-jdk -y

In [1]:
!java -version

openjdk version "11.0.24" 2024-07-16
OpenJDK Runtime Environment (build 11.0.24+8-post-Ubuntu-1ubuntu324.04.1)
OpenJDK 64-Bit Server VM (build 11.0.24+8-post-Ubuntu-1ubuntu324.04.1, mixed mode, sharing)


In [2]:
!readlink -f $(which java)

/usr/lib/jvm/java-11-openjdk-amd64/bin/java


In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"

# Install HBase

In [ ]:
!wget https://downloads.apache.org/hbase/2.5.10/hbase-2.5.10-bin.tar.gz
!tar xvf hbase-2.5.10-bin.tar.gz

In [ ]:
! mv hbase-2.5.10 /opt/hbase

In [4]:
!sed -i 's|# export JAVA_HOME=.*|export JAVA_HOME=/usr/lib/jvm/java-11-openjdk-amd64|' /opt/hbase/conf/hbase-env.sh

In [5]:
# Verify JAVA_HOME
import os

# Check if JAVA_HOME is set, if not set it to the correct path
java_home = '/usr/lib/jvm/java-11-openjdk-amd64'
if not os.path.exists(java_home):
    raise FileNotFoundError(f'Java path not found: {java_home}')
os.environ['JAVA_HOME'] = java_home
os.environ['PATH'] = java_home + '/bin:' + os.environ['PATH']


!java -version

openjdk version "11.0.24" 2024-07-16
OpenJDK Runtime Environment (build 11.0.24+8-post-Ubuntu-1ubuntu324.04.1)
OpenJDK 64-Bit Server VM (build 11.0.24+8-post-Ubuntu-1ubuntu324.04.1, mixed mode, sharing)


In [6]:
#Start HBase
!/opt/hbase/bin/start-hbase.sh

running master, logging to /opt/hbase/bin/../logs/hbase-emma-master-emma-Inspiron-3501.out


In [9]:
!jps

350259 Jps
349730 HMaster


# Install Spark

In [ ]:
!wget -v https://dlcdn.apache.org/spark/spark-3.4.3/spark-3.4.3-bin-hadoop3.tgz

In [ ]:
!tar xvf spark-3.4.3-bin-hadoop3.tgz

In [15]:
!pip3 install findspark --break-system-packages

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
! mv spark-3.4.3-bin-hadoop3 /opt/spark

In [21]:
import os
import findspark

os.environ["SPARK_HOME"] = "/opt/spark"

# Initialize PySpark
findspark.init()


# Collect data

Download csv file from https://obis.org/data/access/#

In [ ]:
import happybase
import csv

connection = happybase.Connection('localhost', port=9090)

try:
    connection.create_table(
        'OBIS',
        {'cf': dict()}  # 'cf' is the column family
    )
except happybase.hbase.ttypes.AlreadyExists:
    print("Table 'OBIS' already exists")

table = connection.table('OBIS')

with open('obis_20230208.csv', 'r') as f:
    reader = csv.reader(f)
    headers = next(reader)
    for row in reader:
        hbase_data = {f'cf:{headers[i]}': row[i] for i in range(1, len(row))}
        # Insert the data into HBase
        table.put(row[0], hbase_data)

connection.close()

In [23]:
#Stop HBase
!/opt/hbase/bin/stop-hbase.sh

stopping hbase...........


In [24]:
!jps

354283 Jps
